In [3]:
import cv2
import mmcv
import tempfile
from mmtrack.apis import inference_mot, init_model

# 输入输出视频路径
input_video = 'demo/demo.mp4'
output = 'outputs/output_demo_MOT_Trace.mp4'

# 指定 config 配置文件 和 模型权重文件，创建模型
mot_config = './configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py'

# 读入待预测视频
imgs = mmcv.VideoReader(input_video)
# 初始化模型
mot_model = init_model(mot_config, device='cuda:0')
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name

# 人数计数
num = 0

# 是否绘制轨迹
TRACE = False

trace_coord_video = []
# 逐帧输入模型预测
for i, img in enumerate(imgs):
    
    ## 运行多目标追踪
    
    result = inference_mot(mot_model, img, frame_id=i)
    
    ## 绘制目标检测框、置信度、类别、ID
    vis_result = mot_model.show_result(
                    img,
                    result,
                    show=False,
                    wait_time=int(1000. / imgs.fps),
                    out_file=None,
                    trace=TRACE)
    
    # 绘制轨迹
    if TRACE:
        img, trace_coord_frame = vis_result[0], vis_result[1]
        
        trace_coord_video.append(trace_coord_frame)

        ## 绘制截止到当前帧，各目标框中心点轨迹

        for trace_coord_frame in trace_coord_video: # 遍历每一帧
            for trace_coord_obj in trace_coord_frame: # 遍历每一个目标
                # 绘制圆，指定圆心坐标和半径，颜色，最后一个参数为线宽，-1表示填充
                cv2.circle(img, (trace_coord_obj[1][0], trace_coord_obj[1][1]),5, trace_coord_obj[1][2], -1)
    # 不绘制轨迹
    else:
        img = vis_result
        
    ##  获取人数
    
    # 用当前最大的 ID 号作为人数值
    if result['track_bboxes'][0][:,0].any():
        num = max(num, 1+max(result['track_bboxes'][0][:,0]))
    else:
        num = num
    # 可视化总人数：图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
    img = cv2.putText(img, str(int(num)), (25, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2)

    # 将处理后的该帧画面图像文件，保存至 /tmp 目录下
    cv2.imwrite(f'{out_path}/{i:06d}.jpg', img)
    
    # 进度条更新
    prog_bar.update()

print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2022-04-07 11:16:59,898 - mmtrack - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth'}
2022-04-07 11:16:59,899 - mmcv - INFO - load model from: https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth
2022-04-07 11:16:59,900 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth
2022-04-07 11:17:00,078 - mmtrack - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmtracking/mot/reid/tracktor_reid_r50_iter25245-a452f51f.pth'}
2022-04-07 11:17:00,080 - mmcv - INFO - load model from: https://download.openmmlab.com/mmtracking/mot/reid/tracktor_reid_r50_iter25245-a452f51f.pth
2022-04-07 11:17:00,081 - mmcv - INFO - load checkpoint from http path: https:/

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8/8, 4.2 task/s, elapsed: 2s, ETA:     0s
 making the output video at outputs/output_demo_MOT_Trace.mp4 with a FPS of 3.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8/8, 21.9 task/s, elapsed: 0s, ETA:     0s
